In [1]:
# import libs 
import pandas as pd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer  

In [2]:
# read the csv
df = pl.read_csv('supervised.csv')

In [3]:
df = df.with_columns(pl.lit(0).alias('target')) # create a dummy target column for splitting 

In [5]:
if 'HSEP001S' in df.columns: # checker 
    print("IN")
else:
    print("false")

IN


In [6]:
# check the target variable cols before imputing invalids 
cols = ['HSHNIAGG', 'HSEP001S']
total = len(df)
for col in cols:
    null_count = df.select(pl.col(col).is_null().sum().alias("nulls")).item() 
    zero_count = df.select((pl.col(col) == 0).sum().alias("zeros")).item()
    negative_count = df.select((pl.col(col) < 0).sum().alias("negatives")).item()
    print(f"Number of nulls for {col}: {null_count} / {total}, {null_count / total * 100 :.2f}")
    print(f"Number of zeroes for {col}: {zero_count} / {total}, {zero_count / total * 100 :.2f}")
    print(f"Number of negatives for {col}: {negative_count} / {total}, {negative_count / total * 100 :.2f}")

Number of nulls for HSHNIAGG: 0 / 853534, 0.00
Number of zeroes for HSHNIAGG: 92709 / 853534, 10.86
Number of negatives for HSHNIAGG: 0 / 853534, 0.00
Number of nulls for HSEP001S: 0 / 853534, 0.00
Number of zeroes for HSEP001S: 92709 / 853534, 10.86
Number of negatives for HSEP001S: 0 / 853534, 0.00


In [7]:
# train test split
X = df.drop('target')
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# impute missing invalid variables for the HSNAIGG, used for making the target variable 
limit = 1.0

# filter out rows where entry equals 0 for computing mean and median 
vals = X_train.filter(pl.col('HSHNIAGG') != 0)
mean_val = vals.select(pl.col('HSHNIAGG').mean()).item()
median_val = vals.select(pl.col('HSHNIAGG').median()).item()
impute_value = mean_val if abs(mean_val - median_val) < limit else median_val # assume median for imputation if distribution is not symmetric 

# replace the 0s with the impute value 
X_train = X_train.with_columns(
    pl.when(pl.col('HSHNIAGG') == 0)
    .then(impute_value)
    .otherwise(pl.col('HSHNIAGG')).alias('HSHNIAGG')
)

X_test = X_test.with_columns(
    pl.when(pl.col('HSHNIAGG') == 0)
    .then(impute_value)
    .otherwise(pl.col('HSHNIAGG')).alias('HSHNIAGG')
)

# check the number of 0s left in the var
zero_count = X_train.select((pl.col('HSHNIAGG') == 0).sum().alias("zeros")).item()
print(f"Number of zeroes for train set: {zero_count} / {len(X_train)}, {zero_count / len(X_train) * 100 :.2f}")

zero_count = X_test.select((pl.col('HSHNIAGG') == 0).sum().alias("zeros")).item() 
print(f"Number of zeroes for test set: {zero_count} / {len(X_test)}, {zero_count / len(X_test) * 100 :.2f}")


Number of zeroes for train set: 0 / 682827, 0.00
Number of zeroes for test set: 0 / 170707, 0.00


In [9]:
if 'HSEP001S' in X_train.columns:
    print("IN")
else:
    print("false")

IN


In [10]:
# check for outliers for the 2 columns for calculating the target and impute them 
# threshold 
threshold = 3.0 
for col in ['HSEP001S', 'HSHNIAGG']: # for both columns 
    train_mean = X_train.select(pl.col(col).mean()).item()
    train_std = X_train.select(pl.col(col).std()).item()
    train_median = X_train.select(pl.col(col).median()).item() 
    
    train_count = len(X_train)
    test_count = len(X_test)
    
    # Add a z score column and compute z score for each entry for that column 
    X_train = X_train.with_columns(((pl.col(col) - train_mean) / train_std).alias(f"{col}_zscore"))
    X_test = X_test.with_columns(((pl.col(col) - train_mean) / train_std).alias(f"{col}_zscore"))
    
    # filter out outliers 
    outliers = X_train.filter(pl.col(f"{col}_zscore").abs() > threshold)
    
    # count the number of outliers compared to the col 
    count_ratio = round(outliers.height / train_count * 100, 2)
    if count_ratio < 10.0 and count_ratio > 0.0: # if there are less than 10% of outliers 
        print(f"Imputing reasonable outliers for {col} for train set: {count_ratio:.2f}%")
        # impute using mean or median 
        impute_val = train_mean if abs(train_mean - train_median) < 1.0 else train_median # assume test distribution follows the train set
        
        X_train = X_train.with_columns(
            pl.when(pl.col(f"{col}_zscore").abs() > threshold)
            .then(impute_val)
            .otherwise(pl.col(col))
            .alias(col)
        )
        
    elif count_ratio >= 10.0: # drop outliers otherwise if the % of outliers is larger than 10% 
        print("Found {col} where number of outliers >= 10% in train set, {count_ratio} %")
        X_train = X_train.filter(pl.col(f"{col}_zscore").abs() <= threshold) # drop rows where the z-score > threshold 


    outliers = X_test.filter(pl.col(f"{col}_zscore").abs() > threshold)
    count_ratio = round(outliers.height / test_count * 100, 2)

    if count_ratio < 10.0 and count_ratio > 0.0:
        print(f"Imputing reasonable outliers for {col} for test set: {count_ratio:.2f}%")

        impute_val = train_mean if abs(train_mean - train_median) < 1.0 else train_median 
        
        X_test = X_test.with_columns(
            pl.when(pl.col(f"{col}_zscore").abs() > threshold)
            .then(impute_val)
            .otherwise(pl.col(col))
            .alias(col)
        )
        
    elif count_ratio >= 10.0:
        print(f"found {col} where number of outliers >= 10% in test set, {count_ratio} %")
        X_test = X_test.filter(pl.col(f"{col}_zscore").abs() <= threshold) # drop rows where the z-score > threshold 
    
    X_train = X_train.drop(f"{col}_zscore") # drop the column after finishing outliers
    X_test = X_test.drop(f"{col}_zscore")

Imputing reasonable outliers for HSEP001S for train set: 0.50%
Imputing reasonable outliers for HSEP001S for test set: 0.51%
Imputing reasonable outliers for HSHNIAGG for train set: 0.58%
Imputing reasonable outliers for HSHNIAGG for test set: 0.59%


In [11]:
# check the number of columns for x train and test 
print(len(X_train.columns))
print(len(X_test.columns))

733
733


In [12]:
# replace the dummy column of 0s; drop the 2 columns used for calculating the target variable
y_train = X_train['HSEP001S'] / X_train['HSHNIAGG']
y_test = X_test['HSEP001S'] / X_test['HSHNIAGG']

In [13]:
# drop the cols used to make the target variable after computing the target variable  
X_train = X_train.drop(['HSEP001S', 'HSHNIAGG'])
X_test = X_test.drop(['HSEP001S', 'HSHNIAGG']) 

In [14]:
# checking if the cols are dropped correctly 
print("True") if ['HSEP001S', 'HSHNIAGG'] not in X_train.columns else print("false")
print("True") if ['HSEP001S', 'HSHNIAGG'] not in X_test.columns else print("false")

True
True


In [15]:
# impute missing (invalid/null) variables for 9 columns 
cols = ['HSWH040S', 'HSWH041S', 'ECYPTAMED', 
        'ECYPMAMED', 'ECYPFAMED', 'ECYHTAMED', 'ECYHMAMED', 'ECYHFAMED', 'ECYMTNMED']
limit = 1.0
for col in cols:
    vals = X_train.filter(pl.col(col).is_not_null()) # filter for non null values 
    mean_val = vals.select(pl.col(col).mean()).item()
    median_val = vals.select(pl.col(col).median()).item()
    impute_value = mean_val if abs(mean_val - median_val) < limit else median_val # assume median for imputation if distribution is not symmetric 

    # replace the nulls with the impute value calculated from the train set  
    X_train = X_train.with_columns(
        pl.when(pl.col(col).is_null())
        .then(impute_value)
        .otherwise(pl.col(col)).alias(col)
    )

    X_test = X_test.with_columns(
        pl.when(pl.col(col).is_null())
        .then(impute_value)
        .otherwise(pl.col(col)).alias(col)
    )
    # check for nulls in the X train and X test
    null_count = X_train.select(pl.col(col).is_null().sum().alias("nulls")).item() 
    print(f"Number of nulls for {col} train set: {null_count} / {len(X_train)}, {null_count / len(X_train) * 100 :.2f}")

    null_count = X_test.select(pl.col(col).is_null().sum().alias("nulls")).item() 
    print(f"Number of nulls for {col} test set: {null_count} / {len(X_test)}, {null_count / len(X_test) * 100 :.2f}")

Number of nulls for HSWH040S train set: 0 / 682827, 0.00
Number of nulls for HSWH040S test set: 0 / 170707, 0.00
Number of nulls for HSWH041S train set: 0 / 682827, 0.00
Number of nulls for HSWH041S test set: 0 / 170707, 0.00
Number of nulls for ECYPTAMED train set: 0 / 682827, 0.00
Number of nulls for ECYPTAMED test set: 0 / 170707, 0.00
Number of nulls for ECYPMAMED train set: 0 / 682827, 0.00
Number of nulls for ECYPMAMED test set: 0 / 170707, 0.00
Number of nulls for ECYPFAMED train set: 0 / 682827, 0.00
Number of nulls for ECYPFAMED test set: 0 / 170707, 0.00
Number of nulls for ECYHTAMED train set: 0 / 682827, 0.00
Number of nulls for ECYHTAMED test set: 0 / 170707, 0.00
Number of nulls for ECYHMAMED train set: 0 / 682827, 0.00
Number of nulls for ECYHMAMED test set: 0 / 170707, 0.00
Number of nulls for ECYHFAMED train set: 0 / 682827, 0.00
Number of nulls for ECYHFAMED test set: 0 / 170707, 0.00
Number of nulls for ECYMTNMED train set: 0 / 682827, 0.00
Number of nulls for ECYMTN

In [16]:
# imputing outliers for all columns  
threshold = 3.0 # threshold 
i = 1
for col in X_train.columns:  
    train_mean = X_train.select(pl.col(col).mean()).item()
    train_std = X_train.select(pl.col(col).std()).item()
    train_median = X_train.select(pl.col(col).median()).item() 
    
    train_count = len(X_train)
    test_count = len(X_test)
    
    # Add a z score column and compute z score for each entry for that column 
    X_train = X_train.with_columns(((pl.col(col) - train_mean) / train_std).alias(f"{col}_zscore"))
    X_test = X_test.with_columns(((pl.col(col) - train_mean) / train_std).alias(f"{col}_zscore"))
    
    # filter out outliers 
    outliers = X_train.filter(pl.col(f"{col}_zscore").abs() > threshold)
    
    # count the number of outliers compared to the col 
    count_ratio = round(outliers.height / train_count * 100, 2)
    if count_ratio < 10.0 and count_ratio > 0.0: # if there are less than 10% of outliers 
        print(f"Imputing reasonable outliers for {i}th col --- {col} for train set: {count_ratio:.2f}%")
        # impute using mean or median 
        impute_val = train_mean if abs(train_mean - train_median) < 1.0 else train_median # assume test distribution follows the train set
        
        X_train = X_train.with_columns(
            pl.when(pl.col(f"{col}_zscore").abs() > threshold)
            .then(impute_val)
            .otherwise(pl.col(col))
            .alias(col)
        )
        
    elif count_ratio >= 10.0: # drop outliers otherwise if the % of outliers is larger than 10% 
        print("Found {i}th col --- {col} where number of outliers >= 10% in train set, {count_ratio} %")
        X_train = X_train.filter(pl.col(f"{col}_zscore").abs() <= threshold) # drop rows where the z-score > threshold 

    outliers = X_test.filter(pl.col(f"{col}_zscore").abs() > threshold)
    count_ratio = round(outliers.height / test_count * 100, 2)

    if count_ratio < 10.0 and count_ratio > 0.0:
        print(f"Imputing reasonable outliers for {i}th col --- {col} for test set: {count_ratio:.2f}%")

        impute_val = train_mean if abs(train_mean - train_median) < 1.0 else train_median 
        
        X_test = X_test.with_columns(
            pl.when(pl.col(f"{col}_zscore").abs() > threshold)
            .then(impute_val)
            .otherwise(pl.col(col))
            .alias(col)
        )
        
    elif count_ratio >= 10.0:
        print(f"found {i}th col --- {col} where number of outliers >= 10% in test set, {count_ratio} %")
        X_test = X_test.filter(pl.col(f"{col}_zscore").abs() <= threshold) # drop rows where the z-score > threshold 
    
    X_train = X_train.drop(f"{col}_zscore") # drop the column after finishing outliers
    X_test = X_test.drop(f"{col}_zscore")
    i += 1

Imputing reasonable outliers for 1th col --- ECYBAS12P for train set: 0.53%
Imputing reasonable outliers for 1th col --- ECYBAS12P for test set: 0.56%
Imputing reasonable outliers for 2th col --- ECYBAS15P for train set: 0.54%
Imputing reasonable outliers for 2th col --- ECYBAS15P for test set: 0.56%
Imputing reasonable outliers for 3th col --- ECYBAS18P for train set: 0.55%
Imputing reasonable outliers for 3th col --- ECYBAS18P for test set: 0.56%
Imputing reasonable outliers for 4th col --- ECYBAS19P for train set: 0.55%
Imputing reasonable outliers for 4th col --- ECYBAS19P for test set: 0.56%
Imputing reasonable outliers for 5th col --- ECYBAS12HP for train set: 0.54%
Imputing reasonable outliers for 5th col --- ECYBAS12HP for test set: 0.56%
Imputing reasonable outliers for 6th col --- ECYBAS15HP for train set: 0.54%
Imputing reasonable outliers for 6th col --- ECYBAS15HP for test set: 0.56%
Imputing reasonable outliers for 7th col --- ECYBAS18HP for train set: 0.55%
Imputing reas

In [ ]:
# standardize pipeline; you can redefine for XGB after linreg 
pipeline = Pipeline([
    ('scaler', StandardScaler()) 
    # add model here 
])

# explicitly standardize target variable 
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1)).ravel()

# standardizing for x only works when you call model.fit() w/ the pipeline  
pipeline.fit(X_train, y_train_scaled)

# No need to manually transform X_test
# y_pred = pipeline.predict(X_test)


# Note: Predict and inverse transform for final regression result 
# y_pred_scaled = pipeline.predict(X_test)
# y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# evaluate metrics using the scaled values, 
# convert the y_pred back to non standardized value using inverse_transform for human understanding